# Decorators

## Motivation

In [300]:
class C:
    # The order is mandatory
    def func():
        print('called func')
    func = staticmethod(func)

In [301]:
c = C()

In [302]:
c.func()

called func


In [303]:
C.func()

called func


In [304]:
class C:
    @staticmethod
    def func():
        print('called func')

In [305]:
c = C()

In [306]:
c.func()

called func


In [307]:
C.func()

called func


## Write a decorator

In [308]:
def hello(func):
    print('hello')

In [309]:
@hello
def add(a, b):
    return a + b

hello


In [310]:
def add(a, b):
    return a + b
add = hello(add)

hello


In [21]:
add(3,4)

TypeError: 'NoneType' object is not callable

In [22]:
def hello(func):
    def proxy(*args, **kwargs):
        print('Hello')
        return func(*args, **kwargs)
    return proxy

In [23]:
@hello
def add(a, b):
    return a + b

In [24]:
add(1,2)

Hello


3

## Parametrized decorator

In [312]:
def say(greetings=None):
    def _say(func):
        def __say(*args, **kwargs):
            print(greetings)
            return func(*args, **kwargs)
        return __say
    return _say

In [313]:
@say('Hola')
def add(a, b):
    return a + b

In [314]:
add(1,2)

Hola


3

In [315]:
add

<function __main__.say.<locals>._say.<locals>.__say(*args, **kwargs)>

In [316]:
@say('Hola')
@say('Adeu')
def add(a, b):
    return a + b

In [317]:
add(1,2)

Hola
Adeu


3

In [318]:
add.__closure__

(<cell at 0x1092a2048: function object at 0x1093dc378>,
 <cell at 0x1092a20d8: str object at 0x109338110>)

In [319]:
# Equivalent code
def add(a, b):
    return a + b
add = say('Hola')(say('Adeu')(add))
add(1,2)

Hola
Adeu


3

In [320]:
add

<function __main__.say.<locals>._say.<locals>.__say(*args, **kwargs)>

In [321]:
add.__closure__

(<cell at 0x109391dc8: function object at 0x1093dc730>,
 <cell at 0x1092a2c48: str object at 0x109338110>)

## Class decorators

In [322]:
def mark(cls):
    cls.new = 42
    return cls

In [323]:
@mark
class A:
    pass

In [324]:
A.new

42

In [325]:
class A:
    pass

A = mark(A)

In [326]:
A.new

42

## Best Practice

`Use wrapps to clarify function origins`

In [327]:
@hello
def add(a, b):
    """Add two objects"""
    return a + b

hello


In [328]:
add(4,5)

TypeError: 'NoneType' object is not callable

In [64]:
add?

Signature: add(*args, **kwargs)
Docstring: <no docstring>
File:      ~/PycharmProjects/AdvancedPython/06-Decorators/<ipython-input-22-cbc183919508>
Type:      function


In [65]:
add.__name__

'proxy'

In [73]:
from functools import wraps

def hello(func):
    @wraps(func)
    def proxy(*args, **kwargs):
        print('Hello')
        return func(*args, **kwargs)
    return proxy

In [74]:
@hello
def add(a, b):
    """Add two objects"""
    return a + b

In [75]:
add?

Signature: add(a, b)
Docstring: Add two objects
File:      ~/PycharmProjects/AdvancedPython/06-Decorators/<ipython-input-74-af8c552ccaba>
Type:      function


In [76]:
add.__name__

'add'

In [78]:
wraps??

Signature: wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
Source:   
def wraps(wrapped,
          assigned = WRAPPER_ASSIGNMENTS,
          updated = WRAPPER_UPDATES):
    """Decorator factory to apply update_wrapper() to a wrapper function

       Returns a decorator that invokes update_wrapper() with the decorated
       function as the wrapper argument and the arguments to wraps() as the
       remaining arguments. Default arguments are as for update_wrapper().
       This is a convenience function to simplify applying partial() to
       update_wrapper().
    """
    return partial(update_wrapper, wrapped=wrapped,
                   assigned=assigned, updated=updated)
File:      ~/miniconda3/envs/py37env/lib/python3.7/functools.py
Type:      function


`Becareful using decorators on recursive functions`

In [85]:
from functools import wraps

def say(greetings=None):
    def _say(func):
        @wraps(func)
        def __say(*args, **kwargs):
            print(greetings)
            return func(*args, **kwargs)
        return __say
    return _say

In [86]:
@say('Hola')
@say('Adeu')
def add(a, b):
    """Add two objects"""
    return a + b

In [87]:
add(4,5)

Hola
Adeu


9

In [88]:
add

<function __main__.add(a, b)>

In [89]:
add.__wrapped__

<function __main__.add(a, b)>

In [92]:
add.__wrapped__.__wrapped__

<function __main__.add(a, b)>

In [93]:
add.__wrapped__.__wrapped__.__wrapped__

AttributeError: 'function' object has no attribute '__wrapped__'

## Advanced decorators

### Type checking

In [99]:
# %load argcheck.py
"""Check function arguments for given type.
"""

import functools


def check(*argtypes):
    """Function argument type checker.
    """

    def _check(func):
        """Takes the function.
        """

        @functools.wraps(func)
        def __check(*args):
            """Takes the arguments
            """
            if len(args) != len(argtypes):
                msg = 'Expected %d but got %d arguments' % (len(argtypes),
                                                            len(args))
                raise TypeError(msg)
            for arg, argtype in zip(args, argtypes):
                if not isinstance(arg, argtype):
                    msg = 'Expected %s but got %s' % (
                        argtypes, tuple(type(arg) for arg in args))
                    raise TypeError(msg)
            return func(*args)
        return __check
    return _check


In [101]:
@check(float, float)
def add(a, b):
    return a + b

In [102]:
add(4, 5)

TypeError: Expected (<class 'float'>, <class 'float'>) but got (<class 'int'>, <class 'int'>)

In [103]:
add(4., 5.)

9.0

## Cached

`Only demontrate the concept. Need for clearing the cache dictionary.`

In [106]:
# %load cached.py
"""Caching results with a decorator.
"""

import functools
import pickle


def cached(func):
    """Decorator that caches.
    """
    cache = {}

    @functools.wraps(func)
    def _cached(*args, **kwargs):
        """Takes the arguments.
        """
        # dicts cannot be use as dict keys
        # dumps are strings and can be used
        key = pickle.dumps((args, kwargs))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return _cached


In [107]:
@cached
def add(a, b):
    print("adding")
    return a + b

In [108]:
add(4, 5)

adding


9

In [109]:
add(4, 5)

9

In [110]:
add(4, b=5)

adding


9

In [111]:
add(5, 4)

adding


9

In [114]:
c = add.__closure__[0].cell_contents

In [115]:
pickle.loads(list(c.keys())[0])

((4, 5), {})

## More caching

In [116]:
from functools import lru_cache

In [119]:
lru_cache?

Signature: lru_cache(maxsize=128, typed=False)
Docstring:
Least-recently-used cache decorator.

If *maxsize* is set to None, the LRU features are disabled and the cache
can grow without bound.

If *typed* is True, arguments of different types will be cached separately.
For example, f(3.0) and f(3) will be treated as distinct calls with
distinct results.

Arguments to the cached function must be hashable.

View the cache statistics named tuple (hits, misses, maxsize, currsize)
with f.cache_info().  Clear the cache and statistics with f.cache_clear().
Access the underlying function with f.__wrapped__.

See:  http://en.wikipedia.org/wiki/Cache_algorithms#Least_Recently_Used
File:      ~/miniconda3/envs/py37env/lib/python3.7/functools.py
Type:      function


In [120]:
@lru_cache(maxsize=2)
def add(a, b):
    print("adding")
    return a + b

In [121]:
add(4, 5)

adding


9

In [123]:
add.cache_info()

CacheInfo(hits=0, misses=1, maxsize=2, currsize=1)

In [124]:
add(4, 5)

9

In [125]:
add.cache_info()

CacheInfo(hits=1, misses=1, maxsize=2, currsize=1)

In [126]:
add(56, 7)

adding


63

In [127]:
add(55, 44)

adding


99

In [128]:
add.cache_info()

CacheInfo(hits=1, misses=3, maxsize=2, currsize=2)

In [129]:
add(4, 5)

adding


9

In [131]:
add.cache_info()

CacheInfo(hits=1, misses=4, maxsize=2, currsize=2)

In [132]:
add.cache_clear()
add.cache_info()

CacheInfo(hits=0, misses=0, maxsize=2, currsize=0)

## Logging

In [137]:
# %load logged.py
"""Helper to switch on and off logging of decorated functions.
"""

from __future__ import print_function

import functools

LOGGING = False


def logged(func):
    """Decorator for logging.
    """

    @functools.wraps(func)
    def _logged(*args, **kwargs):
        """Takes the arguments
        """
        if LOGGING:
            print('logged') # do proper logging here
        return func(*args, **kwargs)
    return _logged


In [138]:
@logged
def add(a, b):
    return a + b

In [139]:
add(4, 5)

9

In [140]:
LOGGING=True

In [141]:
add(4, 5)

logged


9

In [153]:
# %load logged.py
"""Helper to switch on and off logging of decorated functions.
"""

from __future__ import print_function

import functools

LOGGING = True


def logged(func):
    """Decorator for logging.
    """
    if LOGGING:
        @functools.wraps(func)
        def _logged(*args, **kwargs):
            """Takes the arguments
            """
            print('logged') # do proper logging here
            return func(*args, **kwargs)
        return _logged
    return func

In [154]:
@logged
def add(a, b):
    return a + b

In [155]:
add(4, 5)

logged


9

In [156]:
LOGGING=False

In [157]:
add(4, 5)

logged


9

## Registration

In [159]:
# %load registering.py
"""A function registry.
"""

import functools

registry = {}


def register_at_call(name):
    """Register the decorated function at call time.
    """

    def _register(func):
        """Takes the function.
        """

        @functools.wraps(func)
        def __register(*args, **kwargs):
            """Takes the arguments.
            """
            registry.setdefault(name, []).append(func)
            return func(*args, **kwargs)
        return __register
    return _register


def register_at_def(name):
    """Register the decorated function at definition time.
    """

    def _register(func):
        """Takes the function.
        """
        registry.setdefault(name, []).append(func)

        return func
    return _register


`set default method similar to get but...`

In [160]:
e = {}

In [162]:
e.setdefault('a',[]).append(3)

In [163]:
e

{'a': [3]}

In [164]:
e.setdefault('a',[]).append(3)

In [165]:
e

{'a': [3, 3]}

In [168]:
registry

{}

In [169]:
@register_at_def('complex')
def add(a, b):
    return a + b

In [170]:
registry

{'complex': [<function __main__.add(a, b)>]}

In [171]:
add(4, 5)

9

In [172]:
registry

{'complex': [<function __main__.add(a, b)>]}

In [174]:
registry.clear()

In [175]:
registry

{}

In [176]:
@register_at_call('simple')
def add(a, b):
    return a + b

In [178]:
registry

{}

In [179]:
add(4, 5)

9

In [180]:
registry

{'simple': [<function __main__.add(a, b)>]}

In [181]:
add(4, 5)

9

In [182]:
registry

{'simple': [<function __main__.add(a, b)>, <function __main__.add(a, b)>]}

### Assertion in class decorators

In [183]:
assert 1 == 2

AssertionError: 

In [184]:
def assert_fluid(cls):
    assert 0 <= cls.temperature <= 100

In [185]:
@assert_fluid
class Water:
    temperature = 25

In [186]:
@assert_fluid
class Ice:
    temperature = -10

AssertionError: 

### Wrapping example with decorators

In [356]:
def add_special(method_names):
    def _add_special(cls):
        for name in method_names:
            
            def make_meth(name):
            
                def meth(self, *args, **kwargs):
                    orig_meth = getattr(self._wrapped, name)
                    for attr_name in ['__doc__', '__name__']:
                        setattr(meth, attr_name, getattr(orig_meth, attr_name))
                    return orig_meth(*args, **kwargs)
                return meth
            
            setattr(cls, name, make_meth(name))
        return cls
    return _add_special

In [357]:
@add_special(['__len__', '__repr__', '__getitem__'])
class Wrapper:
    def __init__(self, wrapped, allowed):
        self._wrapped = wrapped
        self._allowed = allowed
        
    def __getattr__(self, attr):
        if attr in self._allowed:
            return getattr(self._wrapped, attr)
        raise AttributeError(attr)
    def __repr__(self):
        return repr(self._wrapped)

In [358]:
L = Wrapper([], allowed='append')

In [360]:
L.append(48)

In [361]:
len(L)

1

### Exercice 6.8.1

In [198]:
import timeit
t0 = timeit.default_timer()
timeit.default_timer() - t0

6.483500055765035e-05

In [215]:
from functools import  wraps

def timer(func):
    @wraps(func)
    def _timer(*args, **kwargs):
        t0 = timeit.default_timer()
        _func = func(*args, *kwargs)
        print('Elapsed time: ', timeit.default_timer() - t0)
        return _func
    return _timer

In [216]:
@timer
def add(a, b):
    return a + b

In [217]:
add(4,5)

Elapsed time:  3.632999323599506e-06


9

### Exercice 6.8.2 and 6.8.3

In [380]:
from functools import  wraps

def timer(runs):
    def _timer(func):
        @wraps(func)
        def __timer(*args, **kwargs):
            delta = 0
            for _ in range(runs):
                start = timeit.default_timer()
                _func = func(*args, *kwargs)
                end = timeit.default_timer()
                delta += (end - start)
                print('Result: {0}, Elapsed time:{1}'.format(_func, end - start))
            print('Averaged elapsed time: ', delta/runs)
            return _func
        return __timer
    return _timer

In [381]:
@timer(5)
def add(a, b):
    """Adds two objects"""
    return a + b

In [382]:
add(4,5)

Result: 9, Elapsed time:3.0900009733159095e-06
Result: 9, Elapsed time:1.9070012058364227e-06
Result: 9, Elapsed time:1.5599998732795939e-06
Result: 9, Elapsed time:1.5829991752980277e-06
Result: 9, Elapsed time:1.582000550115481e-06
Averaged elapsed time:  1.944400355569087e-06


9

In [261]:
add?

Signature: add(a, b)
Docstring: Adds two objects
File:      ~/PycharmProjects/AdvancedPython/source/advanced/decorators/<ipython-input-259-14b4f75e0bed>
Type:      function


In [262]:
add.__name__

'add'

### Exercice 6.8.4

In [383]:
from functools import  wraps

TIMING=True

def timer(runs):
    def _timer(func):
        @wraps(func)
        def __timer(*args, **kwargs):
            if TIMING:
                delta = 0
                for i in range(runs):
                    start = timeit.default_timer()
                    _func = func(*args, *kwargs)
                    end = timeit.default_timer()
                    delta += (end - start)
                    print('Result: {0}, Elapsed time:{1}'.format(_func, end - start))
                print('Averaged elapsed time: ', delta/runs)
            else:
                _func = func(*args, **kwargs)
            return _func
        return __timer
    return _timer

In [384]:
@timer(5)
def add(a, b):
    """Adds two objects"""
    return a + b

In [385]:
add(4, 5)

Result: 9, Elapsed time:3.5320008464623243e-06
Result: 9, Elapsed time:2.053999196505174e-06
Result: 9, Elapsed time:1.6590001905569807e-06
Result: 9, Elapsed time:1.6550002328585833e-06
Result: 9, Elapsed time:1.5909990906948224e-06
Averaged elapsed time:  2.098199911415577e-06


9

In [268]:
TIMING=False

In [269]:
add(4, 5)

9

### Exercice 6.8.5

In [401]:
import functools

registry = {}

def register_class(cls):
    """Register the decorated class
    """
#    registry.setdefault('.'.join(cls.__module__ + cls.__module__), cls)
    registry['.'.join((cls.__module__,cls.__module__))] = cls

    return cls

In [402]:
@register_class
class A:
    pass

In [403]:
registry

{'__main__.__main__': __main__.A}

In [408]:
@register_class
class B:
    pass

In [409]:
registry

{'__main__.__main__': __main__.B}

### Extras

In [376]:
import time

@timer(1)
def sleep_loop(n):
    for x in range(n):
        time.sleep(1e-6)

In [379]:
sleep_loop(100_000)

Result: None, Elapsed time:1.8151281379996362
Averaged elapsed time:  1.8151281379996362
